# Class 5: Guest lectures

## [Final Project](https://python-public-policy.afeld.me/en/{{school_slug}}/final_project.ipynb)

- You should have received feedback on your proposal.
- Reminder that it's peer-graded.
   - You should see the notebooks you need to review come through {{lms_name}}.
   - This is an opportunity to see how different people solve different problems.
   - You will lose points if you don't complete your peer grading.

All together, let's make sure our notebooks are properly shared for peer grading:

1. Create a notebook for your Final Project, if you haven't already.
1. [Open the Sharing settings.](https://research.google.com/colaboratory/faq.html#notebook-storage)
1. Under `General access`, change to `LionMail`, then `Commenter`.

## Guest lectures

Encourage keeping your camera on, out of respect for our guests.

### [**Laurie Merrell**](https://www.linkedin.com/in/laurie-merrell/)

> Laurie Merrell is an analytics engineer who works primarily with transit data and [GTFS](https://gtfs.org/) in particular. She decided to pursue a career in data after stints in software quality assurance and higher education administration. To facilitate her career switch, she completed a Master's degree in Analytics at Northwestern University. She is passionate about civic technology and serves on the board of [Chi Hack Night](https://chihacknight.org/), where she founded the [Ghost Buses breakout group](https://ghostbuses.com/).

#### Questions?

### [Alla Goldman Seiffert](https://www.allaseiffert.com/)

> Alla Goldman Seiffert is an attorney, cloud technology advocate, and all-around tech lady who is passionate about making the world a better place. She's currently a Senior Manager of Public Policy at Amazon Web Services (AWS) forcused on public sector cloud adoption. Before this role, Alla was the Director of Cloud Policy and Counsel at Internet Association, where she advocated for their cloud policy priorities and enterprise policy positions. 
>
> Alla also worked for the General Services Administration's (GSA) Technology Transformation Service (TTS) in the Office of Acquisition/18F as the acting director, where she helped government buy technology in a better way.
>
> She has a law degree from the University of Arizona James E. Rogers College of Law.
>
> In her free time, Alla enjoys trying to keep her houseplants alive, dancing, eating dessert, and exploring our country's national parks.

#### Questions?

## Post-{{coding_env_name}}

- {{coding_env_name}} instance (and your copy of files) will be deleted
    - Download things you want to keep, particularly edited notebooks
- Class materials will remain available on [the site](https://python-public-policy.afeld.me/en/{{school_slug}}/) and [through GitHub](https://github.com/afeld/python-public-policy/tree/{{school_slug}})
- [Running notebooks elsewhere](https://python-public-policy.afeld.me/en/{{school_slug}}/resources.html#jupyter-outside-this-course)